<a href="https://colab.research.google.com/github/unniths/Text-Generator-RNN/blob/master/Text_generation_with_an_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Shiva Unnithan
# Text generation with an RNN

In [ ]:
import tensorflow as tf

import numpy as np
import os
import time

In [ ]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt') ## downloading shakespeare

  65536/Unknown - 0s 2us/step

In [ ]:
# Read, then decode for py2 compat
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} character'.format(len(text)))

Length of text: 71041 character


In [ ]:
print(text[:1000]) # printing the first 250 characters

<!DOCTYPE html><html><head><meta name="google" content="notranslate"><meta http-equiv="X-UA-Compatible" content="IE=edge;"><style>@font-face{font-family:'Roboto';font-style:italic;font-weight:400;src:local('Roboto Italic'),local('Roboto-Italic'),url(//fonts.gstatic.com/s/roboto/v18/KFOkCnqEu92Fr1Mu51xIIzc.ttf)format('truetype');}@font-face{font-family:'Roboto';font-style:normal;font-weight:300;src:local('Roboto Light'),local('Roboto-Light'),url(//fonts.gstatic.com/s/roboto/v18/KFOlCnqEu92Fr1MmSU5fBBc9.ttf)format('truetype');}@font-face{font-family:'Roboto';font-style:normal;font-weight:400;src:local('Roboto Regular'),local('Roboto-Regular'),url(//fonts.gstatic.com/s/roboto/v18/KFOmCnqEu92Fr1Mu4mxP.ttf)format('truetype');}@font-face{font-family:'Roboto';font-style:normal;font-weight:700;src:local('Roboto Bold'),local('Roboto-Bold'),url(//fonts.gstatic.com/s/roboto/v18/KFOlCnqEu92Fr1MmWUlfBBc9.ttf)format('truetype');}</style><meta name="referrer" content="origin"><title>Jane Austen___Pri

In [ ]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique character'.format(len(vocab)))

65 unique character


In [ ]:
# Creating a mapping from unique characters to indices 
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [ ]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '$' :   3,
  '&' :   4,
  "'" :   5,
  ',' :   6,
  '-' :   7,
  '.' :   8,
  '3' :   9,
  ':' :  10,
  ';' :  11,
  '?' :  12,
  'A' :  13,
  'B' :  14,
  'C' :  15,
  'D' :  16,
  'E' :  17,
  'F' :  18,
  'G' :  19,
  ...
}


In [ ]:
print('{} ---- characters mapped to int ----> {}'.format(repr(text[:13]), text_as_int[:13]))

'First Citizen' ---- characters mapped to int ----> [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [ ]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

#C reate training examples / target
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

F
i
r
s
t


In [ ]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [ ]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in dataset.take(1):
    print('Input data: ', repr(''. join(idx2char[input_example.numpy()])))
    print('Target data: ', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data:  'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [ ]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 18 ('F')
  expected output: 47 ('i')
Step    1
  input: 47 ('i')
  expected output: 56 ('r')
Step    2
  input: 56 ('r')
  expected output: 57 ('s')
Step    3
  input: 57 ('s')
  expected output: 58 ('t')
Step    4
  input: 58 ('t')
  expected output: 1 (' ')


## Create Training Batches
https://www.tensorflow.org/tutorials/text/text_generation#create_training_batches

In [ ]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset 
# (TF data is design to work with possibly infinite sequences, so it doesn't attempt to shuffle the entire sequence in memory.
# Instead, it maintains a buffer in which is shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

## Build The Model
https://www.tensorflow.org/tutorials/text/text_generation#build_the_model

In [ ]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension 
embedding_dim = 256

# Number of RNN units 
rnn_units = 1024

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units,
                           return_sequences=True,
                           stateful=True,
                           recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [ ]:
model = build_model(
    vocab_size = len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

## Try the Model
https://www.tensorflow.org/tutorials/text/text_generation#try_the_model

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [ ]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (64, None, 256)           16640     
_________________________________________________________________
gru_4 (GRU)                  (64, None, 1024)          3938304   
_________________________________________________________________
dense_4 (Dense)              (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [ ]:
sampled_indices

array([51, 58, 12, 34,  1, 56, 33, 51, 49, 27, 56, 32, 16, 61,  6, 59, 57,
       33, 60, 51, 49, 60, 34, 44, 10, 10, 23,  1,  2, 11, 61, 34, 41,  5,
       16, 61, 38, 50, 26, 21, 39, 54, 10, 16, 26, 45, 60, 48, 59, 33, 39,
       21, 49,  1, 60, 39, 44, 20, 49, 58, 45, 10, 30, 25, 59,  2, 19, 64,
       12, 11, 45, 21, 50, 49, 57, 23, 58, 30, 54,  2,  9,  6, 47, 36, 38,
       40,  4, 33, 63,  5, 46, 13,  1, 32,  3, 55, 48, 16, 54,  7])

In [ ]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 "hts in fellowship\nAnd needly will be rank'd with other griefs,\nWhy follow'd not, when she said 'Tyba"

Next Char Predictions: 
 "mt?V rUmkOrTDw,usUvmkvVf::K !;wVc'DwZlNIap:DNgvjuUaIk vafHktg:RMu!Gz?;gIlksKtRp!3,iXZb&Uy'hA T$qjDp-"


## Train the Model
https://www.tensorflow.org/tutorials/text/text_generation#train_the_model

In [ ]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, "# (base_size, sequence_length, vocab_size)")
print("scalar_loss:       ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 65) # (base_size, sequence_length, vocab_size)
scalar_loss:        4.1749754


In [ ]:
model.compile(optimizer='adam', loss=loss)

## Configure Checkpoints
https://www.tensorflow.org/tutorials/text/text_generation#configure_checkpoints

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoints files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

## Execute the Training
To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [ ]:
EPOCHS=10

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
172/172 [==============================] - 7s 41ms/step - loss: 2.6874
Epoch 2/10
172/172 [==============================] - 7s 41ms/step - loss: 1.9662
Epoch 3/10
172/172 [==============================] - 7s 41ms/step - loss: 1.7005
Epoch 4/10
172/172 [==============================] - 7s 41ms/step - loss: 1.5493
Epoch 5/10
172/172 [==============================] - 7s 41ms/step - loss: 1.4575
Epoch 6/10
172/172 [==============================] - 7s 41ms/step - loss: 1.3985
Epoch 7/10
172/172 [==============================] - 7s 41ms/step - loss: 1.3509
Epoch 8/10
172/172 [==============================] - 7s 41ms/step - loss: 1.3130
Epoch 9/10
172/172 [==============================] - 7s 41ms/step - loss: 1.2780
Epoch 10/10
172/172 [==============================] - 7s 41ms/step - loss: 1.2442


## Generate Text
### Restore the Latest Checkpooint
https://www.tensorflow.org/tutorials/text/text_generation#generate_text

tf.train.latest_checkpoint(checkpoint_dir)

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units,batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [ ]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
gru_5 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_5 (Dense)              (1, None, 65)             66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


## The Prediction Loop

In [ ]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)
  
  # Number of characters to generate
  num_generate = 1000

  # Converting our start to string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures result in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)
    # remove the batch dimension
    predictions = tf.squeeze(predictions, 0)

    # using a categorical distribution to predict the character returned by the model
    predictions + predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    # We pass the predicted character as the next input to the model
    # along with previous hidden state
    input_eval = tf.expand_dims([predicted_id], 0)

    text_generated.append(idx2char[predicted_id])
  
  return (start_string +''.join(text_generated))

In [ ]:
print(generate_text(model, start_string=u'ROMEO: '))

ROMEO: hunowndle son;
It shall go suns to read. You know
Shall you hence, I may come to the outward things; Edward,
You may not proclaim an oat, being
Here it was my father, service in scott the open'maver, Arge, so heavy
As banishment seme and beast; no time hath here prevails
To hole against him, both to flint pain again.

FRANCISCAnd, fort, the price of Warwick, though tears,
When he grant me to his, because we shall speak.
Go, born; as we may beseech thee this: why
nch.

DUCHESS OF YORK:

HORTENSIO:
Beseech you, brother,
You at my fact, good mother, tell me that
I may proceed. O, that you all Montague,
No more than that his woes be found.

GREMIO:
O, my lord; 'twesp al any julm too;
Forbid in a low all appointeded wretch.

WARWICK:
Why, then, that serve his love?

LEONTES:
Tragu,
You know how to brain aftainst;' 'cause' such as you
are, lector, my lord and your crown,
And thou didst seek the pitch of your downs and broph is guilthom, in so subalting.
Adieu! I mevolio, who had born'

The easiest thing you can do to improve the results it to train it for longer (try EPOCHS=30).

You can also experiment with a different start string, or try adding another RNN layer to improve the model's accuracy, or adjusting the temperature parameter to generate more or less random predictions.

## Advanced: Customized Training 

The above training procedure is simple, but does not give you much control.

So now that you've seen how to run the model manually let's unpack the training loop, and implement it ourselves. This gives a starting point if, for example, to implement curriculum learning to help stabilize the model's open-loop output.



In [ ]:
model = build_model(
    vocab_size = len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [ ]:
optimizer = tf.keras.optimizers.Adam()

In [ ]:
@tf.function
def train_step(inp, target):
  with tf.GradientTape() as tape:
    predictions = model(inp)
    loss = tf.reduce_mean(
        tf.keras.losses.sparse_categorical_crossentropy(
            target, predictions, from_logits=True))
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    return loss
